In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.core.operations import SymmOp
from pymatgen.util.coord_utils import get_angle

In [2]:
class Polymer(object):
    def __init__(self, start_monomer, s_head, s_tail, 
                 monomer, head, tail, 
                 end_monomer, e_head, e_tail, n_units, link_distance=1.0):
        """
        Args:
            monomer (Molecule): The monomer
            n_units (int): number of monomer units excluding the start and terminal molecules
            head (int): index of the atom in the monomer that forms the head
            tail (int): tail atom index. monomers will be connected from head to tail
            link_distance (float): distance between consecutive monomers
        """
        self.start = s_head
        self.end = s_tail
        self.monomer = monomer
        self.n_units = n_units
        self.link_distance = link_distance
        #translate monomers so that head atom is at the origin
        start_monomer.translate_sites(range(len(start_monomer)), - monomer.cart_coords[s_head])
        monomer.translate_sites(range(len(monomer)), - monomer.cart_coords[head])
        end_monomer.translate_sites(range(len(end_monomer)), - monomer.cart_coords[e_head])
        self.mon_vector = monomer.cart_coords[tail] - monomer.cart_coords[head]
        self.moves = {1:[1,0,0], 2:[0,1,0], 3:[0,0,1], 4:[-1,0,0], 5:[0,-1,0], 6:[0,0,-1]}
        self.prev_move = 1
        #places the start monomer at the beginning of the chain
        self.molecule = start_monomer.copy()
        self.length = 0
        # create the chain
        self._create()
        # terminate the chain with the end_monomer
        while self.length != self.n_units+1:
            end_mon_vector = end_monomer.cart_coords[e_tail] - end_monomer.cart_coords[e_head]
            self._add_monomer(end_monomer, end_mon_vector)
    
    def _create(self):
        """
        create the polymer from the monomer by random walk
        """
        while self.length != self.n_units:
            monomer = self.monomer.copy()
            self._add_monomer(monomer, self.mon_vector)
            
    def _next_move_direction(self):
        """
        pick a move at random from the list of moves
        """
        move = np.random.randint(1, 7)
        while self.prev_move == (move + 3)%6:
            move = np.random.randint(1, 7)
        self.prev_move = move
        print "move", move
        return np.array(self.moves[move])
    
    def _align_monomer(self, monomer, mon_vector, move_direction):
        """
        rotate the monomer so that it is aligned along the move direction
        """
        axis = np.cross(mon_vector, move_direction)
        origin = monomer[self.start].coords
        angle = get_angle(mon_vector, move_direction)
        #print "axis, origin, angle", axis, origin, angle
        op = SymmOp.from_origin_axis_angle(origin, axis, angle)
        monomer.apply_operation(op)

    def _add_monomer(self, monomer, mon_vector):
        """
        extend the polymer molecule by adding a monomer along mon_vector direction
        """
        move_direction = self._next_move_direction()
        translate_by = self.molecule.cart_coords[self.end] + self.link_distance * move_direction
        monomer.translate_sites(range(len(monomer)), translate_by)
        self._align_monomer(monomer, mon_vector, move_direction)
        # add monomer if there are no crossings
        does_cross = False
        for i, site in enumerate(monomer):
            try:
                self.molecule.append(site.specie, site.coords)
            except:
                does_cross = True
                polymer_length = len(self.molecule)
                self.molecule.remove_sites(range(polymer_length-i, polymer_length))
                break
        if not does_cross:
            self.length += 1
            self.end += len(self.monomer)
            print "length: ", self.length       

In [3]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
s_head = 0
s_tail = 5
# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
head = 0
tail = 4
# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
e_head = 0
e_tail = 4

n_units = 100
link_distance = 3.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

move 2
length:  1
move 3
length:  2
move 2
length:  3
move 1
length:  4
move 2
length:  5
move 2
length:  6
move 6
length:  7
move 1
length:  8
move 6
length:  9
move 2
length:  10
move 2
length:  11
move 4
length:  12
move 5
length:  13
move 3
length:  14
move 4
length:  15
move 6
length:  16
move 6
length:  17
move 5
length:  18
move 1
length:  19
move 6
length:  20
move 1
length:  21
move 2
length:  22
move 4
length:  23
move 6
length:  24
move 2
length:  25
move 6
length:  26
move 1
length:  27
move 5
length:  28
move 5
length:  29
move 6
length:  30
move 1
length:  31
move 1
length:  32
move 1
length:  33
move 5
length:  34
move 5
length:  35
move 5
length:  36
move 1
length:  37
move 6
length:  38
move 3
move 4
length:  39
move 3
move 3
move 1
move 2
length:  40
move 1
length:  41
move 3
length:  42
move 5
move 3
length:  43
move 5
length:  44
move 4
length:  45
move 2
length:  46
move 3
length:  47
move 4
length:  48
move 5
length:  49
move 1
length:  50
move 5
length:  51
move 

In [4]:
# write the polymer molecule
peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [5]:
from pymatgen.io.babel import BabelMolAdaptor

peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol
peo_optimized.to(filename="polymer_optimized.xyz", fmt="xyz")

Get the polymer topology: atoms, bonds, angles and dihedrals

In [6]:
import openbabel

obmol = peo_obabel.openbabel_mol

atoms = [x.GetIdx()-1 for x in openbabel.OBMolAtomIter(obmol)]
bonds = [[x.GetBeginAtomIdx()-1, x.GetEndAtomIdx()-1] 
         for x in openbabel.OBMolBondIter(obmol)] 
angles = [list(x) for x in openbabel.OBMolAngleIter(obmol)]
dihedrals = [list(x) for x in openbabel.OBMolTorsionIter(obmol)]

print len(atoms), len(bonds), len(angles), len(dihedrals)

716 614 720 410
